In [1]:
import pandas as pd


### Download the dataframe from FED parsing file

In [2]:
file = pd.read_csv("df_min_press_speech.csv")
file.head()

,date,text
0,2012-01-06,"It is the start of a new year, the traditional..."
1,2012-01-06,\r\n Thank you for the opportunity to sp...
2,2012-01-07,\r\n Thank you and happy New Year. It is...
3,2012-01-13,\r\n It's a pleasure to be here this mor...
4,2012-01-16,\r\n It is certainly a pleasure to be he...


### Create functions to find negative and positive words 

P.S. all the functions called negative, but they were also used for positive words

In [3]:
def Find_all_freq(tokens,all_word,stopword):
    
    # all words in file content 
    freq_all = Counter(tokens)
    
    # delete words that are not in lm_dictionary and words in stopwords and not in stopwords
    filtered_word = list(set(tokens) & set(all_word) - set(stopword))
    freq_all_filtered = {key: value for key, value in freq_all.items() if key in filtered_word}
    
    return freq_all_filtered

In [4]:
def Find_neg_freq(tokens,freq_all_filtered,negative_list,all_lm_word, stopword):
    filtered_word = list(set(tokens) & set(all_lm_word) - set(stopword))
    neg_filtered_word = list(set(filtered_word) & set(negative_list))
    freq_neg_filtered = {key: value for key, value in freq_all_filtered.items() if key in neg_filtered_word}
    return freq_neg_filtered

def negative_pro(neg,all_word):
    num_neg = sum(neg.values())
    num_all_word = sum(all_word.values())
    neg_pro = num_neg/num_all_word
    return neg_pro

def Matrix1(f_neg,i,matrix):
    matrix1 = matrix.copy()
    mat = pd.DataFrame(f_neg.items(), columns=['neg',str(i)])
    mat = mat.set_index('neg') 
    matrix1 = pd.concat([matrix1, mat],axis=1)
    return matrix1

def Matrix2(matrix):
    matrix2 = matrix.copy()
    matrix2[matrix2>0] = 1
    return matrix2

def Matrix3(num_word_list,freq):
    num_word = sum(freq.values())  
    num_word_list.append(num_word)
    return num_word_list

#### Negative words matrices

In [5]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import re
import pandas as pd

def Main():
    
    nltk.download('stopwords')
    stopword = stopwords.words('english')
    
    # lm
    lm_dictionary = pd.read_csv('/Users/aidosutebayev/Documents/NYU/7871NLP/Dictionary/Loughran-McDonald_MasterDictionary_1993-2023.csv')
    all_lm_word = lm_dictionary['Word'].to_list()
    lm_Negative = lm_dictionary[lm_dictionary['Negative']!=0]
    lm_Negative = lm_Negative['Word'].to_list()
    lm_pro_list = []
    
    #tfidf
    matrix1 = pd.DataFrame()
    num_word_list = []
    
    filing_date_list = []
    
    count = 0
    index_list = file.index.tolist()
    for i in index_list:
        try:
            file_content = file.loc[i,'text'].upper()
            tokens = re.findall('\w+', file_content)
            filing_date_list.append(file.loc[i,'date'])


            # Get target data_lm
            freq_all = Find_all_freq(tokens,all_lm_word,stopword)
            freq_neg_lm = Find_neg_freq(tokens,freq_all,lm_Negative,all_lm_word,stopword)

            # normal proportion of negative words_lm
            lm_pro = negative_pro(freq_neg_lm,freq_all)
            lm_pro_list.append(lm_pro)

            # build tfidf_lm
            matrix1 = Matrix1(freq_neg_lm,count,matrix1)
            num_word_list = Matrix3(num_word_list,freq_all)

            count+=1 

        except:
            print('empty')

    matrix1.fillna(0, inplace=True)
        
    result_list_lm = [matrix1, num_word_list, lm_pro_list,filing_date_list]
    return result_list_lm

result_list_lm = Main()
result_list_lm.append(Matrix2(result_list_lm[0]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aidosutebayev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_36687/1563293868.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matrix1.fillna(0, inplace=True)


In [6]:
result_list_lm

[                0    1    2    3    4    5    6    7    8    9  ...  879  880  \
 neg                                                             ...             
 DRAG          4.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0   
 DOWNTURN      2.0  0.0  1.0  1.0  2.0  0.0  1.0  0.0  1.0  0.0  ...  0.0  0.0   
 CONTRACTION   2.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 THREATENED    1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 DEPRESSED     1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  5.0  0.0  ...  0.0  0.0   
 ...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 EXAGGERATING  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 MISDIRECTED   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 EXAGGERATION  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0   
 JEOPARDIZED   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
 OUTAGE        0

#### Positive word matrices

In [7]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import re
import pandas as pd

def Main():
    
    nltk.download('stopwords')
    stopword = stopwords.words('english')
    
    # lm
    lm_dictionary = pd.read_csv('/Users/aidosutebayev/Documents/NYU/7871NLP/Dictionary/Loughran-McDonald_MasterDictionary_1993-2023.csv')
    all_lm_word = lm_dictionary['Word'].to_list()
    lm_Negative = lm_dictionary[lm_dictionary['Positive']!=0]
    lm_Negative = lm_Negative['Word'].to_list()
    lm_pro_list = []
    
    #tfidf
    matrix1 = pd.DataFrame()
    num_word_list = []
    
    filing_date_list = []
    
    count = 0
    index_list = file.index.tolist()
    for i in index_list:
        try:
            file_content = file.loc[i,'text'].upper()
            tokens = re.findall('\w+', file_content)
            filing_date_list.append(file.loc[i,'date'])


            # Get target data_lm
            freq_all = Find_all_freq(tokens,all_lm_word,stopword)
            freq_neg_lm = Find_neg_freq(tokens,freq_all,lm_Negative,all_lm_word,stopword)

            # normal proportion of negative words_lm
            lm_pro = negative_pro(freq_neg_lm,freq_all)
            lm_pro_list.append(lm_pro)

            # build tfidf_lm
            matrix1 = Matrix1(freq_neg_lm,count,matrix1)
            num_word_list = Matrix3(num_word_list,freq_all)

            count+=1 

        except:
            print('empty')

    matrix1.fillna(0, inplace=True)
        
    result_list_lm = [matrix1, num_word_list, lm_pro_list,filing_date_list]
    return result_list_lm

result_list_lm_pos = Main()
result_list_lm_pos.append(Matrix2(result_list_lm_pos[0]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aidosutebayev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_36687/1053115037.py:51: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matrix1.fillna(0, inplace=True)


#### Negative words tfidf dataframe

In [9]:
mat1 = result_list_lm[0].to_numpy()
mat2 = result_list_lm[4].to_numpy()
mat3 = result_list_lm[1]
N = len(mat1[0])
df = mat2.sum(axis=1)

import numpy as np
from numpy import inf
part1 = 1+np.log(mat1)
part2 = 1+np.log(mat3)
part3 = np.log(N/df)

part1[np.isneginf(part1)] = 0
part2[np.isneginf(part2)] = 0
part3[np.isneginf(part3)] = 0

part3 = part3[:,np.newaxis]
tfidf = np.divide(part1,part2)*part3
result_tfidf = tfidf.sum(axis=0)
result_LM = pd.DataFrame(list(zip(result_tfidf,result_list_lm[2], result_list_lm[3])),
columns =['tfidf_neg','proportion', 'date'])

result_LM = result_LM[['tfidf_neg','proportion','date']]  # Replace with your desired column names

/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_36687/3882081548.py:9: RuntimeWarning: divide by zero encountered in log
  part1 = 1+np.log(mat1)


In [14]:
result_LM.head()

,tfidf_neg,proportion,date
0,35.766943,0.042488,2012-01-06
1,18.595948,0.024059,2012-01-06
2,35.496525,0.058704,2012-01-07
3,27.907909,0.036066,2012-01-13
4,10.571655,0.015971,2012-01-16


#### Positive words tfidf dataframe

In [11]:
mat1_pos = result_list_lm_pos[0].to_numpy()
mat2_pos = result_list_lm_pos[4].to_numpy()
mat3_pos = result_list_lm_pos[1]
N_pos = len(mat1_pos[0])
df_pos = mat2_pos.sum(axis=1)

import numpy as np
from numpy import inf
part1_pos = 1+np.log(mat1_pos)
part2_pos = 1+np.log(mat3_pos)
part3_pos = np.log(N_pos/df_pos)

part1_pos[np.isneginf(part1_pos)] = 0
part2_pos[np.isneginf(part2_pos)] = 0
part3_pos[np.isneginf(part3_pos)] = 0

part3_pos = part3_pos[:,np.newaxis]
#part3 = part3.T
#part1 = part1.T
tfidf_pos = np.divide(part1_pos,part2_pos)*part3_pos
result_tfidf_pos = tfidf_pos.sum(axis=0)
result_LM_pos = pd.DataFrame(list(zip(result_tfidf_pos,result_list_lm_pos[2], result_list_lm_pos[3])),
columns =['tfidf_pos','proportion', 'date'])

# result_LM['filing_date'] = pd.to_datetime(result_LM['filing_date'])
# result_LM['filing_date'] = result_LM['filing_date'].dt.strftime("%Y-%m-%d")
# Specify the columns you want to keep in the final DataFrame
result_LM_pos = result_LM_pos[['tfidf_pos','proportion','date']]  # Replace with your desired column names

/var/folders/9x/srv3h8gx32z9h7gqd2pbh94m0000gn/T/ipykernel_36687/572766382.py:9: RuntimeWarning: divide by zero encountered in log
  part1_pos = 1+np.log(mat1_pos)


#### Merge 2 dataframes into 1

In [12]:
result_LM_total = pd.merge(result_LM['tfidf_neg'], result_LM_pos[['tfidf_pos', 'date']], left_index=True, right_index=True)

#### Save dataframe to csv file

In [13]:
result_LM_total.to_csv('dictionary_tfidf.csv')